In [1]:
import pyqrcode
import cv2
from pyzbar.pyzbar import decode
import webbrowser as wb
import os
from PIL import Image
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import datetime


import numpy as np


import pytesseract



In [2]:

# Função para encontrar QR Code em uma imagem e extrair o link ;
def extrair_link_qr_code(imagem):
    # Converter imagem para escala de cinza
    imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    # Aplicar filtro para melhorar o contraste
    imagem_filtrada = cv2.bilateralFilter(imagem_cinza, 11, 17, 17)
    # Localizar QR Code na imagem
    qr_codes = decode(imagem_filtrada)
    if len(qr_codes) > 0:
        # Extrair o link do QR Code
        link = qr_codes[0].data.decode('utf-8')
        return link
    else:
        return None


    
    
#Função top do cabeludo scalpelando o site da receita federal ;
def extrai_texto_nfe(url):
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')

    tr_list = soup.find_all('tr')

    text_list = []

    for tr_list_item in tr_list:
        text_list.append(tr_list_item.text)
  
    result_list = []

    for text_list_item in text_list:
        new_list = text_list_item.split('\n')
    

        new_text_list = []
     
        new_text_list.append(new_list[2])
        new_text_list.append(new_list[6])
        new_text_list.append(new_list[8])
        new_text_list.append(new_list[10])
        new_text_list.append(new_list[12])
        print(new_text_list)
        result_list.append(new_text_list)
    return result_list

#Funções para tratar os dados raspados

def produt(lista):
    
    global produto
    produto = []
    for prod in range(len(lista)):
        prdt = lista[prod][0]
        
        
        produto.append(prdt)

    
        
###

def trat_quantid(lista):
    
    global quantidade
    quantidade = []
    for prod in range(len(lista)):
        qtde = lista[prod][1].replace("Qtde.:","")
        qtde2 = qtde.replace(",",".")
        qtde3 = float(qtde2)
        
        quantidade.append(qtde3)

      
        
###

def trat_unidade(lista):
    global unidade
    unidade = []
    for prod in range(len(lista)):
        unid = lista[prod][2].replace("UN: ","")
        
        unidade.append(unid)


      
        
###

def trat_valor_unid(lista):
    global valor_unidade
    valor_unidade = []
    for prod in range(len(lista)):
        vlun = lista[prod][3].replace("Vl. Unit.:","")
        vlun2 = vlun.replace(",",".")
        vlun3 = float(vlun2)
        
        valor_unidade.append(vlun3)

       
        
        
###

def trat_valor_total(lista):
    global valor_total
    valor_total = []
    for prod in range(len(lista)):
        vltt = lista[prod][4].replace("Vl. Total","")
        vltt2 = vltt.replace(",",".")
        vltt3 = float(vltt2)
        
        valor_total.append(vltt3)

   
    
    
#Função pra preencher o DataFrame  !!! ATENÇÃO - O DF tem que ser criado anteriormente (no caso "compras2")
#- RECOMENDAÇÃO: Criaro DF dentro da função ???

def montar_df(lista):
    compras2['VALOR_TOTAL'] = valor_total
    compras2['VALOR_UNITARIO'] = valor_unidade
    compras2['UNIDADE'] = unidade
    compras2['QUANTIDADE'] = quantidade
    compras2['PRODUTO'] = produto


    
### FUNÇÃO SUPREMA ANINHANDO TODAS AS OUTRAS: ;


def extrair_df(lista):
    global compras2
    compras2 = pd.DataFrame(columns = ['PRODUTO', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITARIO', 'VALOR TOTAL'])    
    trat_quantid(lista)
    trat_unidade(lista)
    trat_valor_total(lista)
    trat_valor_unid(lista)
    produt(lista)
    montar_df(lista)
    dia = datetime.date.today()


    compras2.to_csv(f'C:\\Users\\kimge\\OneDrive\\Alecrim\\notas\\qrcode{dia}.csv')
    
df_resultado = pd.DataFrame()   
def adicionar_dados(df_existente, compras2):
    global df_resultado
    df_resultado = pd.concat([df_existente, compras2])
    return df_resultado

In [3]:
for roots, dirs, files in os.walk(r'C:\Users\kimge\OneDrive\Alecrim\notas\Notas'):
    for file in files:
        imagem = cv2.imread(f"C:\\Users\\kimge\\OneDrive\\Alecrim\\notas\\Notas\\{file}")
        nota = extrair_link_qr_code(imagem)
        lista_compras = extrai_texto_nfe(nota)
        extrair_df(lista_compras)
        df_final = adicionar_dados(df_resultado, compras2)
        df_final.to_csv(f'C:\\Users\\kimge\\OneDrive\\Alecrim\\notas\\compras_acumuladas.csv')
        df_final

['PIMENTAO REI VERMELHO 150GR', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa05,83', 'Vl. Total5,83']
['PIMENTAO REI AMARELO 150G', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa05,83', 'Vl. Total5,83']
['GUARD FLOR LILAS 18X19,5 1UN', 'Qtde.:40', 'UN: UN', 'Vl. Unit.:\xa00,9', 'Vl. Total36,00']
['#SACOLA CAMISETA BRANCA 58X70X0,036 PROM', 'Qtde.:2', 'UN: UN', 'Vl. Unit.:\xa00,25', 'Vl. Total0,50']
['BERINJELA KG PROM', 'Qtde.:1,185', 'UN: KG', 'Vl. Unit.:\xa04,5', 'Vl. Total5,33']
['ABOBRINHA KG PROM', 'Qtde.:1,36', 'UN: KG', 'Vl. Unit.:\xa04,89', 'Vl. Total6,65']
['MANJERICAO UN PROM', 'Qtde.:2', 'UN: UN', 'Vl. Unit.:\xa01,25', 'Vl. Total2,50']
['LIMAO THAITI KG PROM', 'Qtde.:1,165', 'UN: KG', 'Vl. Unit.:\xa02,99', 'Vl. Total3,48']
['BETERRABA KG PROM', 'Qtde.:1,02', 'UN: KG', 'Vl. Unit.:\xa04,5', 'Vl. Total4,59']
['CENOURA KG PROM', 'Qtde.:1,06', 'UN: KG', 'Vl. Unit.:\xa03,78', 'Vl. Total4,01']
['CACHACA MARIA GUARDA CL700ML', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa031,68', 'Vl. Total31,68']
['AGUARD CANINH